# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 27 2022 10:42AM,247370,10,8578345,Eywa Pharma Inc.,Released
1,Sep 27 2022 10:42AM,247370,10,8578346,Eywa Pharma Inc.,Released
2,Sep 27 2022 10:42AM,247370,10,8578347,Eywa Pharma Inc.,Released
3,Sep 27 2022 10:42AM,247370,10,8578348,Eywa Pharma Inc.,Released
4,Sep 27 2022 10:42AM,247370,10,8578349,Eywa Pharma Inc.,Released
5,Sep 27 2022 10:42AM,247370,10,8578350,Eywa Pharma Inc.,Released
6,Sep 27 2022 10:42AM,247370,10,8578352,Eywa Pharma Inc.,Released
7,Sep 27 2022 10:40AM,247372,19,ACG-2102491362,ACG North America LLC,Released
8,Sep 27 2022 10:39AM,247368,10,PRF-37906,Bio-PRF,Released
9,Sep 27 2022 10:39AM,247368,10,PRF-37917,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,247365,Released,1
31,247368,Released,13
32,247370,Released,7
33,247371,Released,1
34,247372,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247365,NaN,NaN,1.0
247368,NaN,NaN,13.0
247370,NaN,NaN,7.0
247371,NaN,NaN,1.0
247372,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247201,0.0,0.0,1.0
247202,0.0,0.0,1.0
247205,0.0,0.0,1.0
247250,0.0,0.0,1.0
247258,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247201,0,0,1
247202,0,0,1
247205,0,0,1
247250,0,0,1
247258,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247201,0,0,1
1,247202,0,0,1
2,247205,0,0,1
3,247250,0,0,1
4,247258,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247201,,,1
1,247202,,,1
2,247205,,,1
3,247250,,,1
4,247258,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.
7,Sep 27 2022 10:40AM,247372,19,ACG North America LLC
8,Sep 27 2022 10:39AM,247368,10,Bio-PRF
21,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc."
22,Sep 27 2022 9:58AM,247365,10,Eye Pharma Inc
23,Sep 27 2022 9:56AM,247364,10,"Methapharm, Inc."
31,Sep 27 2022 9:56AM,247363,10,Bio-PRF
32,Sep 27 2022 9:54AM,247205,21,"NBTY Global, Inc."
33,Sep 27 2022 9:54AM,247201,21,"NBTY Global, Inc."
34,Sep 27 2022 9:54AM,247202,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,,,7
1,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,,,1
2,Sep 27 2022 10:39AM,247368,10,Bio-PRF,,,13
3,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",,,1
4,Sep 27 2022 9:58AM,247365,10,Eye Pharma Inc,,,1
5,Sep 27 2022 9:56AM,247364,10,"Methapharm, Inc.",,,8
6,Sep 27 2022 9:56AM,247363,10,Bio-PRF,,,1
7,Sep 27 2022 9:54AM,247205,21,"NBTY Global, Inc.",,,1
8,Sep 27 2022 9:54AM,247201,21,"NBTY Global, Inc.",,,1
9,Sep 27 2022 9:54AM,247202,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,7,,
1,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,1,,
2,Sep 27 2022 10:39AM,247368,10,Bio-PRF,13,,
3,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",1,,
4,Sep 27 2022 9:58AM,247365,10,Eye Pharma Inc,1,,
5,Sep 27 2022 9:56AM,247364,10,"Methapharm, Inc.",8,,
6,Sep 27 2022 9:56AM,247363,10,Bio-PRF,1,,
7,Sep 27 2022 9:54AM,247205,21,"NBTY Global, Inc.",1,,
8,Sep 27 2022 9:54AM,247201,21,"NBTY Global, Inc.",1,,
9,Sep 27 2022 9:54AM,247202,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,7,,
1,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,1,,
2,Sep 27 2022 10:39AM,247368,10,Bio-PRF,13,,
3,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",1,,
4,Sep 27 2022 9:58AM,247365,10,Eye Pharma Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,7,NaN,NaN
1,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,1,NaN,NaN
2,Sep 27 2022 10:39AM,247368,10,Bio-PRF,13,NaN,NaN
3,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",1,NaN,NaN
4,Sep 27 2022 9:58AM,247365,10,Eye Pharma Inc,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,7,0.0,0.0
1,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,1,0.0,0.0
2,Sep 27 2022 10:39AM,247368,10,Bio-PRF,13,0.0,0.0
3,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",1,0.0,0.0
4,Sep 27 2022 9:58AM,247365,10,Eye Pharma Inc,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3462817,104,7.0,2.0
15,494707,82,24.0,0.0
19,1236630,40,3.0,0.0
20,494555,18,16.0,18.0
21,988858,4,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3462817,104,7.0,2.0
1,15,494707,82,24.0,0.0
2,19,1236630,40,3.0,0.0
3,20,494555,18,16.0,18.0
4,21,988858,4,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,104,7.0,2.0
1,15,82,24.0,0.0
2,19,40,3.0,0.0
3,20,18,16.0,18.0
4,21,4,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,15,Released,82.0
2,19,Released,40.0
3,20,Released,18.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,2.0,0.0,0.0,18.0,0.0
Executing,7.0,24.0,3.0,16.0,0.0
Released,104.0,82.0,40.0,18.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,2.0,0.0,0.0,18.0,0.0
1,Executing,7.0,24.0,3.0,16.0,0.0
2,Released,104.0,82.0,40.0,18.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,2.0,0.0,0.0,18.0,0.0
1,Executing,7.0,24.0,3.0,16.0,0.0
2,Released,104.0,82.0,40.0,18.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()